In [ ]:
# The URL of the MISP instance to connect to
misp_url = 'http://127.0.0.1:8080'
# Can be found in the MISP web interface under 
# http://+MISP_URL+/users/view/me -> Authkey
misp_key = 'LBelWqKY9SQyG0huZzAMqiEBl6FODxpgRRXMsZFu'
# Should PyMISP verify the MISP certificate
misp_verifycert = False

# Getting the API key (automatically generated on the trainig VM)

In [ ]:
from pathlib import Path

api_file = Path('apikey')
if api_file.exists():
    misp_url = 'http://127.0.0.1'
    misp_verifycert = False
    with open(api_file) as f:
        misp_key = f.read().strip()
    print(misp_key)

# Initialize PyMISP

In [ ]:
from pymisp import PyMISP

misp = PyMISP(misp_url, misp_key, misp_verifycert, debug=False)

# Index Search (fast, only returns events metadata)

## Search unpublished events

**WARNING**: By default, the search query will only return all the events listed on the index page

In [ ]:
r = misp.search_index(published=False)
print(r)

## Get the meta data of events

In [ ]:
r = misp.search_index(eventid=[17217, 1717, 1721, 17218])

## Search Tag & mix with other parameters

In [ ]:
r = misp.search_index(tags=['tlp:white'])
for e in r:
    print(e)

In [ ]:
r = misp.search_index(tag='TODO:VT-ENRICHMENT', published=False)

In [ ]:
r = misp.search_index(tag=['!TODO:VT-ENRICHMENT', 'tlp:white'], published=False)  # ! means "not this tag"

## Full text search on event info field

In [ ]:
r = misp.search_index(eventinfo='circl')

## Search in the values of each attributes

In [ ]:
r = misp.search_index(attribute='8.8.8.8')
print(r)

## Search by org

In [ ]:
r = misp.search_index(org='CIRCL')

## Search updated events

In [ ]:
r = misp.search_index(timestamp='1h')
print(r)

# Search full events (Slower, returns full events)

## Getting timestamps

In [ ]:
from datetime import datetime, date, timedelta
from dateutil.parser import parse

int(datetime.now().timestamp())

d = parse('2018-03-24')
int(d.timestamp())

today = int(datetime.today().timestamp())
yesterday = int((datetime.today() - timedelta(days=1)).timestamp())

print(today, yesterday)



In [ ]:
r = misp.search(values='8.8.8.8')
print(r)

In [ ]:
r = misp.search(not_values='8.8.8.8')

In [ ]:
r = misp.search(category='Payload delivery')

In [ ]:
r = misp.search(values='8.8.8.8', metadata=True) # no attributes

In [ ]:
r = misp.search(timestamp=['2h', '1h'])

In [ ]:
r = misp.search(values='8.8.8.8', enforceWarninglist=True)

In [ ]:
r = misp.search(values='8.8.8.8', deleted=True)

In [ ]:
r = misp.search(values='8.8.8.8', publish_timestamp=1521846000)  # everything published since that timestamp

In [ ]:
r = misp.search(values='8.8.8.8', last='1d')  # everything published in the last <interval>

In [ ]:
r = misp.search(values='8.8.8.8', timestamp=[yesterday, today])  # everything updated since that timestamp

In [ ]:
r = misp.search(values='8.8.8.8', withAttachments=True)  # Return attachments

# Search for attributes

In [ ]:
r = misp.search(controller='attributes', value='8.8.8.9')

In [ ]:
r = misp.search(controller='attributes', value='wrapper.no', event_timestamp='5d')  # only consider events updated since this timestamp

In [ ]:
r

## Because reason

In [ ]:
tag_to_remove = 'foo'

events = misp.search(tags=tag_to_remove, pythonify=True)

for event in events:
    for tag in event.tags:
        if tag.name == tag_to_remove:
            print(f'Got {tag_to_remove} in {event.info}')
            misp.untag(event.uuid, tag_to_remove)
            break
    for attribute in event.attributes:
        for tag in attribute.tags:
            if tag.name == tag_to_remove:
                print(f'Got {tag_to_remove} in {attribute.value}')
                misp.untag(attribute.uuid, tag_to_remove)
                break

In [ ]:
logs = misp.search_logs(model='Tag', title='tlp:white')
print(logs)

In [ ]:
logs = misp.search_logs(model='Event', pythonify=True)
#print(logs)
for l in logs:
    print(l.title)

In [ ]:
log = misp.search_logs(model='Tag', title=tag_to_remove)[0]
roles = misp.get_roles_list()
for r in roles:
    if r['Role']['name'] == 'User':
        new_role = r['Role']['id']
        break
user = misp.get_user(log['Log']['user_id'])
user['User']['role_id'] = new_role
misp.edit_user(user['User']['id'], **user['User'])